In [1]:
from parrot import Parrot
import pandas as pd
from tqdm import tqdm
import os
import nlpaug.augmenter.word as naw
# Download word2vec model
os.environ["MODEL_DIR"] = '/workspaces/mlops-template-Shunian-Chen/model'
model_dir = os.environ["MODEL_DIR"]
# from nlpaug.util.file.download import DownloadUtil
# DownloadUtil.download_word2vec(dest_dir=model_dir) 

2022-12-15 12:47:16.970349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 12:47:17.170331: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-15 12:47:17.871349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-15 12:47:17.871480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

In [3]:
def paraphrase_aug(data, num):
    aug_text = {'text': []}
    for text in data:
        paraphrases = parrot.augment(text,  max_length = 250, max_return_phrases = 5, do_diverse=False)
        if paraphrases:
            for para in paraphrases:
                if para[1] != 0:
                    aug_text['text'].append(para[0])
                    num -= 1
    df_aug = pd.DataFrame(aug_text)
    return df_aug, num

def data_augs(data, num, model):
    aug_text = {'text': []}

    if model == 'word_embs':
        aug = naw.WordEmbsAug(model_type='word2vec', model_path=model_dir+'GoogleNews-vectors-negative300.bin',action="substitute", device='cuda')
    elif model == 'contextual':
        aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute", device='cuda')
    elif model == 'synonym':
        aug = naw.SynonymAug(aug_src='wordnet')

    for text in data.values.tolist():
        augmented_text = aug.augment(str(text))
        aug_text['text'] += augmented_text
        num -= 1
        if num == 0:
            break
    df_aug = pd.DataFrame(aug_text)

    return df_aug, num


In [4]:
origin_data = pd.read_csv("mbti_train.csv")

In [5]:
cnt = origin_data.groupby("label").count().reset_index()
avg = len(origin_data) // 16
print(cnt, avg)

    label   text
0       0   7639
1       1   6279
2       2  55830
3       3  40742
4       4  12630
5       5   9950
6       6  69372
7       7  48976
8       8   3391
9       9   1731
10     10  25672
11     11  26415
12     12   1507
13     13   1631
14     14   7311
15     15   8752 20489


In [6]:
aug = pd.DataFrame(cnt)
aug["to_aug"] = cnt['text'].apply(lambda x: (avg - x)).clip(0)
aug.columns = ["label", "length", "to_aug"]
aug

,label,length,to_aug
0,0,7639,12850
1,1,6279,14210
2,2,55830,0
3,3,40742,0
4,4,12630,7859
5,5,9950,10539
6,6,69372,0
7,7,48976,0
8,8,3391,17098
9,9,1731,18758


In [7]:
def augment_data_by_label(data, num, label):
    print("data: ", data.shape)
    augmented_data, num = paraphrase_aug(data, num)
    new_data = pd.concat([pd.DataFrame(data), augmented_data], axis=0)
    print("num after paraphrase: ", num)
    print("augmented_data: ", augmented_data.shape)
    print("new_data: ", new_data.shape)
    if num > 0:
        augmented_data, num = data_augs(new_data, num, 'synonym')
        new_data = pd.concat([new_data, augmented_data], axis=0)
        print("num after synonym: ", num)
        print("augmented_data: ", augmented_data.shape)
    if num > 0:
        augmented_data, num = data_augs(new_data, num, 'contextual')
        new_data = pd.concat([new_data, augmented_data], axis=0)
        print("num after contextual: ", num)
        print("augmented_data: ", augmented_data.shape)
    # if num > 0:
    #     augmented_data, num = data_augs(new_data, num, label, 'word_embs')
    #     new_data = pd.concat([new_data, augmented_data], axis=0)
    print("new_data: ", new_data.shape)
    new_data['label'] = label
    return new_data


In [8]:
def data_augmentation(data, aug):
    to_aug = aug[aug['to_aug'] > 0]['label'].tolist()
    data_augmented = data
    for label in to_aug:
        num = aug[aug['label'] == label]['to_aug'].values.tolist()[0]
        print(f"Augmenting label: {label}, num to aug: {num}")

        label_data = data[data['label'] == label].reset_index(drop=True)
        augmented_data = augment_data_by_label(label_data['text'], num, label)
        
        data_augmented = pd.concat([data_augmented, augmented_data], axis=0)
        data_augmented.to_csv(f"mbti_train_aug{label}.csv", index=False)
    return data_augmented

In [9]:
augmented_data = data_augmentation(origin_data, aug)

Augmenting label: 0, num to aug: 12850
data:  (7639,)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_30448/3638840456.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_30448/3638840456.py'                        │
│                                                                                                  │
│ /tmp/ipykernel_30448/3921323624.py:9 in data_augmentation                                        │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_30448/3921323624.py'                        │
│                                                                                                  │
│ /tmp/ipykernel_30448/336250232.py:3 in augment_data_by_label                                     │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_30448/336250232.py'                         │
│                                                                                                  │
│ /tmp/ipykernel_30448/2781927735.py:4 in paraphrase_aug                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_30448/2781927735.py'                        │
│                                                                                                  │
│ /home/codespace/venv/lib/python3.8/site-packages/parrot/parrot.py:112 in augment                 │
│                                                                                                  │
│   109 │   │   │     early_stopping=True,                                                         │
│   110 │   │   │     num_return_sequences=max_return_phrases)                                     │
│   111 │     else:                                                                                │
│ ❱ 112 │   │   preds = self.model.generate(                                                       │
│   113 │   │   │   │   input_ids,                                                                 │
│   114 │   │   │   │   do_sample=True,                                                            │
│   115 │   │   │   │   max_length=max_length,                                                     │
│                                                                                                  │
│ /home/codespace/venv/lib/python3.8/site-packages/torch/autograd/grad_mode.py:27 in               │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/codespace/venv/lib/python3.8/site-packages/transformers/generation_utils.py:1351 in        │
│ generate                                                   

In [ ]:
augmented_data.to_csv("mbti_train_augmented.csv", index=False)

In [ ]:
origin_data.shape

(327828, 2)

In [ ]:
augmented_data.shape

(388749, 2)